In [2]:
import os
import numpy as np
import pickle as pk

with open('gym_2d.pkl','rb') as f:
    data = pk.load(f)
print(data.keys())

dict_keys(['split', 'annotations'])


In [3]:
i = 200
print(data['annotations'][i].keys())
print(data['annotations'][i]['label'])
print(data['annotations'][i]['total_frames'])
print(data['annotations'][i]['keypoint'].shape)
# Data Shape: (1, 24, 17, 2)
# people cnt, frame cnt, keypoint cnt, x & y

# Target Shape: (40091, 3, 50, 25, 2)
# datapoints, x & y & z, frame cnt, keypoint cnt, people cnt

dict_keys(['frame_dir', 'label', 'img_shape', 'original_shape', 'total_frames', 'keypoint', 'keypoint_score'])
43
24
(1, 24, 17, 2)


In [4]:
# Data Shape: (datapoints, 2, 50, 17, 2)
# Target Shape: (datapoints, 3, 50, 25, 2)

In [21]:
train_count = len(data['split']['train'])
val_count = len(data['split']['val'])
assert train_count + val_count == len(data['annotations'])

max_body = 2
num_joint = 17
max_frame = 300
train_data = np.zeros((train_count, 2, max_frame, num_joint, max_body))
val_data = np.zeros((val_count, 2, max_frame, num_joint, max_body))
train_label = np.zeros((train_count))
val_label = np.zeros((val_count))

In [22]:
train_cnt = 0
val_cnt = 0
for instance in data['annotations']:
    label_ins = int(instance['label'])
    kpts_ins = instance['keypoint']
    kpts_ins = np.transpose(kpts_ins, (3, 1, 2, 0))
    kpts_ins = kpts_ins[:, 0:max_frame, :, :]

    if instance['frame_dir'] in data['split']['train']:
        train_data[train_cnt, :, 0:kpts_ins.shape[1], :, :] = kpts_ins
        train_label[train_cnt] = label_ins
        train_cnt += 1
    else:
        val_data[val_cnt, :, 0:kpts_ins.shape[1], :, :] = kpts_ins
        val_label[val_cnt] = label_ins
        val_cnt += 1

In [23]:
os.makedirs('data', exist_ok=True)
np.save('data/train_data.npy', train_data)
np.save('data/val_data.npy', val_data)
np.save('data/train_label.npy', train_label)
np.save('data/val_label.npy', val_label)

In [24]:
train_label = list(map(int, list(train_label)))
val_label = list(map(int, list(val_label)))
with open('data/train_label.pkl', 'wb') as f:
    pk.dump([[], train_label], f)
with open('data/val_label.pkl', 'wb') as f:
    pk.dump([[], val_label], f)